## 1) Libraries Installation
##### The cell below is to help you keep track the libraries used and install them quickly.
##### Ensure the correct library names are used, and follow the syntax: **%pip install PACKAGE_NAME**.

In [213]:
%pip install pandas
%pip install matplotlib
%pip install arch 
%pip install scikit-learn

# add commented pip installation lines for packages used as shown above for ease of testing
# the line should follow the format %pip install PACKAGE_NAME

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 2) Main Section for Code
### **ALL code for machine learning and dataset analysis** should be entered below.
##### Ensure that your code is clear and readable.
##### Remember to include comments and markdown notes as necessary to explain and highlight important segments of your code.

In [214]:
## libraries 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg
from arch import arch_model
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns




### Data Manipulation

#### Cleaning and transformation

In [288]:
# load data
quar_df = pd.read_csv('Quarterly Data.csv', index_col= [0])
quar_df.columns = [c.upper() for c in quar_df.columns]
factors = quar_df.loc["factors"]
tcodes = quar_df.loc["transform"]

quar_df.head()




,GDPC1,PCECC96,PCDGX,PCESVX,PCNDX,GPDIC1,FPIX,Y033RC1Q027SBEAX,PNFIX,PRFIX,...,TNWMVBSNNCBBDIX,TLBSNNBX,TLBSNNBBDIX,TABSNNBX,TNWBSNNBX,TNWBSNNBBDIX,CNCFX,S&P 500,S&P DIV YIELD,S&P PE RATIO
sasdate,,,,,,,,,,,,,,,,,,,,,
factors,0.000,0.000,1.0000,1.0000,1.0000,0.000,0.0000,1.0000,1.0000,1.0000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,1.0000,0.0000,0.0000
transform,5.000,5.000,5.0000,5.0000,5.0000,5.000,5.0000,5.0000,5.0000,5.0000,...,2.00,5.00,1.00,5.00,5.00,2.00,5.0000,5.0000,2.0000,5.0000
3/1/1959,3352.129,2039.017,68.6651,1374.1739,689.1172,354.894,357.0174,47.8021,171.0783,282.9707,...,1341.67,333245.24,266668.11,2426352.41,2092.54,1674.48,124.9663,55.5167,3.1765,18.6211
6/1/1959,3427.667,2070.508,71.2495,1394.7096,694.8197,382.500,368.0640,49.2332,176.0004,292.8479,...,1294.15,345845.01,273329.36,2434975.21,2088.55,1650.63,126.5305,57.5067,3.1012,19.2899
9/1/1959,3430.057,2092.138,72.6270,1413.6627,697.4699,357.798,371.9585,50.8372,180.7406,287.3778,...,1320.72,354808.05,275139.48,2434604.03,2079.21,1612.34,128.9557,58.7300,3.0720,18.9544


We choose to use only variables included in Stock and Watson's factor consideration as these variables were proven to have predictive power for economies. Moreover, doing so eliminates almost half of all variables, making further computation and processing easier.

Source: https://www.princeton.edu/~mwatson/papers/Stock_Watson_HOM_Vol2.pdf

In [290]:
print(factors.value_counts())

factors
1.0    125
0.0    120
Name: count, dtype: int64


In [289]:
# remove non-SW factors 
columns_to_remove = quar_df.columns[factors == 0]
quar_df= quar_df.drop(columns=columns_to_remove)
quar_df_filter = quar_df.iloc[2:]


# process dates
quar_df_filter = quar_df_filter.loc[pd.notna(quar_df_filter.index), :]
quar_df_filter.index = pd.date_range(start="1959-01-01", freq="Q", periods=len(quar_df_filter))
quar_df_filter.index = quar_df_filter.index.to_period("Q")

quar_df_filter.head()



C:\Users\User\AppData\Local\Temp\ipykernel_6808\2571391808.py:9: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  quar_df_filter.index = pd.date_range(start="1959-01-01", freq="Q", periods=len(quar_df_filter))


,PCDGX,PCESVX,PCNDX,Y033RC1Q027SBEAX,PNFIX,PRFIX,A014RE1Q156NBEA,A823RL1Q225SBEA,FGRECPTX,SLCEX,...,SPCS20RSA,TWEXAFEGSMTHX,EXUSEU,EXSZUSX,EXJPUSX,EXUSUKX,EXCAUSX,UMCSENTX,USEPUINDXM,S&P 500
1959Q1,68.6651,1374.1739,689.1172,47.8021,171.0783,282.9707,0.8,-7.7,556.0802,531.4011,...,NaN,NaN,NaN,4.3161,359.8417,2.8095,0.9706,NaN,NaN,55.5167
1959Q2,71.2495,1394.7096,694.8197,49.2332,176.0004,292.8479,1.4,7.6,578.9494,532.5791,...,NaN,NaN,NaN,4.3190,359.8417,2.8145,0.9619,95.3,NaN,57.5067
1959Q3,72.6270,1413.6627,697.4699,50.8372,180.7406,287.3778,0.1,5.5,564.7381,532.8908,...,NaN,NaN,NaN,4.3164,359.9281,2.8083,0.9545,NaN,NaN,58.7300
1959Q4,69.1573,1430.8157,701.7617,50.7342,180.2333,279.2502,0.8,-3.9,566.5192,530.0971,...,NaN,NaN,NaN,4.3344,360.2305,2.8025,0.9498,93.8,NaN,57.7633
1960Q1,71.3815,1443.7405,703.5338,52.4985,186.4967,287.8417,2.1,-14.2,618.3418,539.0478,...,NaN,NaN,NaN,4.3338,360.5337,2.8030,0.9520,100.0,NaN,56.2767


In [291]:
# transformation function
def apply_transformation(series, tcode):
    if tcode == 1:
        return series
    elif tcode == 2:
        return series.diff()
    elif tcode == 3:
        return series.diff().diff()
    elif tcode == 4:
        return np.log(series)
    elif tcode == 5:
        return np.log(series).diff()
    elif tcode == 6:
        return np.log(series).diff().diff()
    elif tcode == 7:
        return series.pct_change()
    else:
        raise ValueError(f"Unknown TCODE: {tcode}")
    
tcodes = pd.read_csv("Quarterly Data.csv", nrows=1, index_col=0)
tcodes.columns = [c.upper() for c in tcodes.columns]

# transform the time series
quar_df_t = quar_df.apply(lambda x: apply_transformation(x, tcodes[x.name].item()))
quar_df_t.head()


,PCDGX,PCESVX,PCNDX,Y033RC1Q027SBEAX,PNFIX,PRFIX,A014RE1Q156NBEA,A823RL1Q225SBEA,FGRECPTX,SLCEX,...,SPCS20RSA,TWEXAFEGSMTHX,EXUSEU,EXSZUSX,EXJPUSX,EXUSUKX,EXCAUSX,UMCSENTX,USEPUINDXM,S&P 500
sasdate,,,,,,,,,,,,,,,,,,,,,
factors,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0,1.0,1.0000,1.0000,...,1.0,1.0,1.0,1.0000,1.0000,1.0000,1.0000,1.0,1.0,1.0000
transform,5.0000,5.0000,5.0000,5.0000,5.0000,5.0000,1.0,1.0,5.0000,5.0000,...,5.0,5.0,5.0,5.0000,5.0000,5.0000,5.0000,1.0,2.0,5.0000
3/1/1959,68.6651,1374.1739,689.1172,47.8021,171.0783,282.9707,0.8,-7.7,556.0802,531.4011,...,NaN,NaN,NaN,4.3161,359.8417,2.8095,0.9706,NaN,NaN,55.5167
6/1/1959,71.2495,1394.7096,694.8197,49.2332,176.0004,292.8479,1.4,7.6,578.9494,532.5791,...,NaN,NaN,NaN,4.3190,359.8417,2.8145,0.9619,95.3,NaN,57.5067
9/1/1959,72.6270,1413.6627,697.4699,50.8372,180.7406,287.3778,0.1,5.5,564.7381,532.8908,...,NaN,NaN,NaN,4.3164,359.9281,2.8083,0.9545,NaN,NaN,58.7300


#### Feature Selection

In [292]:
# remove highly correlated features

""" corr_matrix = quar_df_t.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
quar_df_t.drop(columns=to_drop, inplace=True) """


# recursive feature elimination

""" model = LinearRegression()
rfe = RFE(model, n_features_to_select=10)
X_selected = rfe.fit_transform(quar_df_t, quar_df_t.iloc[:, 0])
quar_df_t = pd.DataFrame(X_selected, index=quar_df_t.index) """


# select by importance

selector = SelectFromModel(RandomForestRegressor(n_estimators=100))
X_selected = selector.fit_transform(quar_df_t, quar_df_t.iloc[:, 0])
quar_df_t = pd.DataFrame(X_selected, index=quar_df_t.index)


# arima order selection
def select_arima_order(series):
    p_values = range(0, 6)
    d_values = range(0, 3)  # Differencing term
    q_values = range(0, 6)  # Moving average term
    best_aic, best_order = float('inf'), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                try:
                    model = ARIMA(series, order=(p, d, q)).fit()
                    if model.aic < best_aic:
                        best_aic, best_order = model.aic, (p, d, q)
                except:
                    continue
    return best_order

best_arima_order = select_arima_order(quar_df_t.iloc[:, 0])

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
c:\Users\User\AppData\Local\Programs\

### Modeling

#### AR

In [293]:
ar_model = AutoReg(quar_df_t.iloc[:, 0], lags=best_arima_order[0]).fit()
ar_forecast = ar_model.predict(start=len(quar_df_t), end=len(quar_df_t) + 2)

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa

#### ARIMA

In [294]:
arima_model = ARIMA(quar_df_t.iloc[:, 0], order=best_arima_order).fit()
arima_forecast = arima_model.get_forecast(steps=3)
arima_forecast_mean = arima_forecast.predicted_mean
arima_forecast_conf_int = arima_forecast.conf_int()


c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
c:\Users\User\AppData\Local\Programs\

#### GARCH

In [295]:
garch_model = arch_model(quar_df_t.iloc[:, 0], vol='Garch', p=1, q=1).fit()
garch_forecast = garch_model.forecast(horizon=3).mean.iloc[-1]

Iteration:      1,   Func. Count:      6,   Neg. LLF: 8018.922630062351
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1944.0948107320578
Iteration:      3,   Func. Count:     18,   Neg. LLF: 1944.0290205291337
Iteration:      4,   Func. Count:     23,   Neg. LLF: 1943.7107794959081
Iteration:      5,   Func. Count:     28,   Neg. LLF: 1942.3631395055036
Iteration:      6,   Func. Count:     33,   Neg. LLF: 1940.9757696482238
Iteration:      7,   Func. Count:     38,   Neg. LLF: 1940.099753054512
Iteration:      8,   Func. Count:     43,   Neg. LLF: 1939.3108475198708
Iteration:      9,   Func. Count:     48,   Neg. LLF: 1934.705987263911
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1897.4202659749028
Iteration:     11,   Func. Count:     58,   Neg. LLF: 2487.5370216008278
Iteration:     12,   Func. Count:     64,   Neg. LLF: 1823.7970738532167
Iteration:     13,   Func. Count:     69,   Neg. LLF: 1820.7877816002328
Iteration:     14,   Func. Count:     75,   Neg. LLF: 

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.949e+05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.1 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


### Evaluation

#### Model Evaluation

In [296]:
def directional_accuracy(actual, forecast):
    direction_actual = np.sign(actual.diff().dropna())
    direction_forecast = np.sign(forecast.diff().dropna())
    return accuracy_score(direction_actual, direction_forecast)

print(f"AR Directional Accuracy: {directional_accuracy(quar_df_t.iloc[:, 0], ar_forecast):.2%}")
print(f"ARIMA Directional Accuracy: {directional_accuracy(quar_df_t.iloc[:, 0], arima_forecast_mean):.2%}")
print(f"GARCH Directional Accuracy: {directional_accuracy(quar_df_t.iloc[:, 0], garch_forecast):.2%}")


ValueError: Found input variables with inconsistent numbers of samples: [264, 2]

#### Graphs